In [4]:
import tensorflow as tf
from tensorflow import keras

#define linear regression model using tf.keras.Model
class LinearRegression(tf.keras.Model):
  def __init__(self):
    super(LinearRegression, self).__init__()
    self.linear_layer = tf.keras.layers.Dense(1, activation=None)

  def call(self, x):
    y_pred = self.linear_layer(x)

    return y_pred

#define loss function
@tf.function
def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.square(y_pred - y)) #MSE loss function => mean{(y'-y)^2}

#define gradient descent optimizer
optimizer = tf.optimizers.SGD(0.01)

#define folder address to save tensorboard summary information
summary_writer = tf.summary.create_file_writer('./tensorboard_log')

#define function for optimization
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = mse_loss(y_pred, y)

  #log tensorboard at every step
  with summary_writer.as_default():
    tf.summary.scalar('loss', loss, step=optimizer.iterations)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

#set input and output values
x_train = [1.0, 2.0, 3.0, 4.0]
y_train = [2.0, 4.0, 6.0, 8.0]

#get data in batch form using tf.data API
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().batch(1)
train_data_iter = iter(train_data)

#define LinearRegression model
LinearRegression_model = LinearRegression()

#perform gradient descent 1000 times
for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter)
  batch_xs = tf.expand_dims(batch_xs, 0) #expansion of dimension to meet 2-dimension (the minimun imput dimension of tf.keras.layers.Dense API)
  train_step(LinearRegression_model, batch_xs, batch_ys)

#set input data for test
x_test = [3.5, 5.0, 5.5, 6.0]
test_data = tf.data.Dataset.from_tensor_slices((x_test))
test_data = test_data.batch(1)

for batch_x_test in test_data:
  batch_x_test = tf.expand_dims(batch_x_test, 0)
  print(tf.squeeze(LinearRegression_model(batch_x_test), 0).numpy())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
[6.997228]
[9.982493]
[10.977582]
[11.972671]
